# CNN Avec peu de données

#### Chargement du code

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

np.random.seed(123)  # for reproducibility
tf.random.set_seed(123)

%load_ext autoreload
%autoreload 2

In [ ]:
from TP3_utils import load_dataset

DATA_PATH = 'Data'
CATEGORIES = ["accordion", "anchor", "barrel", "binocular"]

(X_train, Y_train), (X_test, Y_test), num_classes = load_dataset(DATA_PATH, CATEGORIES)
data_shape = X_train[0].shape

print("Training data shape:", data_shape)
print("Training labels shape:", Y_train.shape)
print("Test data shape:", X_test.shape)
print("Test labels shape:", Y_test.shape)
print("Number of classes:", num_classes)

In [ ]:
import time
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from TP3_utils import CNN, affiche, eval_classif
# from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# lr_schedulerBis = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
# earlStopBis = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, restore_best_weights=True, verbose = 1, mode="min", start_from_epoch=20)

lr=1e-4
batch_size=min([X_train.shape[0], 256])
epochs=16
ad= Adam(learning_rate=lr)

model = CNN(data_shape)
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=ad,
    metrics=['accuracy']
)

tps1 = time.time()
history =model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, Y_test)
#     callbacks=[earlStopBis, lr_schedulerBis],
)
tps2 = time.time()

# Evaluation
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

affiche(history)
preds = model.predict(X_test)
eval_classif(Y_test, preds)
print("Temps d'entraînement : {:.2f} secondes".format(tps2 - tps1))

In [ ]:
from TP3_utils import data_augmentation

inputs = tf.keras.Input(shape=data_shape)
x = data_augmentation(inputs)
x = CNN(data_shape)(x)
model_aug = tf.keras.Model(inputs, x, name="CNN_with_Augmentation")

model_aug.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

# Entraînement
history =model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, Y_test)
#     callbacks=[earlStopBis, lr_schedulerBis],
)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

affiche(history)
preds = model.predict(X_test)
eval_classif(Y_test, preds)
print("Temps d'entraînement : {:.2f} secondes".format(tps2 - tps1))


In [ ]:
from tensorflow.keras.layers import Input
from TP3_utils import MLP_transfer

# Transfer learning
input_tensor = Input(shape=(224,224,3))
VGG = tf.keras.applications.VGG16(weights='imagenet',
    include_top=False,
    input_tensor=input_tensor
)
for layer in VGG.layers:
    layer.trainable = False

model_transfer = tf.keras.Sequential([
    tf.keras.layers.Input(shape=VGG.output_shape[1:]),
    MLP_transfer(VGG.output_shape[1:], num_classes)
])
model_transfer.summary()

model_transfer.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-5),
    metrics=['accuracy']
)

tps1 = time.time()
history =model_transfer.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, Y_test)
#     callbacks=[earlStopBis, lr_schedulerBis],
)
tps2 = time.time()

################## Fine-tuning #################

for layer in VGG.layers[-4:]:
    layer.trainable = True

model_transfer.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-6),
    metrics=['accuracy']
)

tps1 = time.time()
history =model_transfer.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, Y_test)
#     callbacks=[earlStopBis, lr_schedulerBis],
)
tps2 = time.time()

loss, accuracy = model_transfer.evaluate(X_test, Y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

affiche(history)
preds = model_transfer.predict(X_test)
eval_classif(Y_test, preds)
print("Temps d'entraînement : {:.2f} secondes".format(tps2 - tps1))


In [ ]:
from TP3_utils import load_mnist_with_noise
(X_train, Y_train, X_train_noise), (X_test, Y_test, X_test_noise) = load_mnist_with_noise()

# Display the train data and a version of it with added noise
for i in range(5):
    plt.subplot(2,5,i+1)
    plt.imshow(X_train[i,:].reshape([28,28]), cmap='gray')
    plt.axis('off')
    plt.subplot(2,5,i+6)
    plt.imshow(X_train_noise[i,:].reshape([28,28]), cmap='gray')
    plt.axis('off')
    plt.show()

ImportError: cannot import name 'load_mnist_with_noise' from 'TP3_utils' (/Users/joachim/gws/TPs_Rob/TP_ML/TP3/TP3_utils.py)

In [ ]:
from TP3_utils import auto_encoder

model_ae = auto_encoder(data_shape)

model_ae.compile(
    loss="mean_square_error",
    optimizer=ad,
    metrics=['accuracy']
)

history =model_ae.fit(
    X_train,
    Y_train,
    batch_size=32,
    epochs=20,
    verbose=1,
    validation_data=(X_test, Y_test)
#     callbacks=[earlStopBis, lr_schedulerBis],
)

loss, accuracy = model_ae.evaluate(X_test, Y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

affiche(history)
preds = model_ae.predict(X_test)
eval_classif(Y_test, preds)
print("Temps d'entraînement : {:.2f} secondes".format(tps2 - tps1))

In [ ]:
from TP3_utils import vae, loss_func

model_vae, mu, log_variance = vae(data_shape)

model_vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss=loss_func(mu, log_variance))
model_vae.fit(X_train, X_train, epochs=70, batch_size=32, shuffle=True, validation_data=(X_test, X_test))